In [40]:
import os
import json
import re
from operator import itemgetter

import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import words
from nltk.corpus import wordnet as wn
from nltk.corpus import framenet as fn
from nltk.corpus import stopwords
from nltk.corpus.reader.framenet import PrettyList
from nltk.stem.snowball import SnowballStemmer
from time import time
# ...
# nltk.download('words')
# nltk.download('stopwords')
# nltk.download('averaged_perceptron_tagger')
# nltk.download('punkt')

In [ ]:
directory_summary_main = r"data/BBC News Summary/News Articles/"
sub_dirs = os.listdir(directory_summary_main)

word_count_dict = {}
root_word_count_dict = {}

total_token_count = 0
max_file_count = 1000

i = 0

for sub in sub_dirs:
    files = os.listdir(directory_summary_main + sub)
    for f in files:
        try:
            text = open(directory_summary_main + sub + "/" + f, "r").read()
            tokens = word_tokenize(text.lower())
            total_token_count = total_token_count + len(tokens)
            for tok in tokens:
                if tok.isalpha():
                    if tok not in word_count_dict:
                        word_count_dict[tok] = 0
                    word_count_dict[tok] = word_count_dict[tok] + 1
            i = i + 1
        except:
            continue

        #     if i % 1000==1:
        #         print(i)
        #     if i > max_file_count:
        #         break

word_freq_dict = {}
for w in word_count_dict:
    word_freq_dict[w] = word_count_dict[w]/total_token_count

word_freq_dict_val = {k: v for k, v in sorted(word_freq_dict.items(), key=lambda x: x[1], reverse= False)}  

with open("word_fequency.json", "w") as outfile:
    json.dump(word_freq_dict_val, outfile)
    
word_freq_dict_val

In [ ]:
# event = {"calamity": {"famine", "plague", "tidal_wave", "tsunami" , "Avalanche", "Blizzard", "Earthquake", "Fire", "Flood", "Heat wave", "Hurricane", "Landslide", "Lightning strike", "Volcanic eruption"}}

In [49]:
def get_event_typing(w, thres = 0.0, m = 3, depth = 3):
    ws = wn.synsets(w)
    
    hyper = lambda s: s.hypernyms()
    types1 = {}
    types2 = {}

    for e in ws:
        w_tmp = e.name().split(".")[0]
        if w_tmp==w:
            t = 0
            c = 0 
            i = 0
            tmp_types = ""
            
            tmp_event_types = list(e.closure(hyper))
            
            for tmp_event in tmp_event_types:
                d = wn.path_similarity(e, tmp_event)
                tmp_event_name = tmp_event.name().split(".")[0]
                
                b = m**i
                t = t + (d/b)
                types2[tmp_event_name] = d
                
                if d and (d >= (thres/t)):
                    tmp_types = tmp_types + ":" + tmp_event_name
                    if c >= depth - 1:
                        break
                    c = c + 1
                    
                i = i + 1
                
            if len(tmp_types) > 0:
                types1[tmp_types] = t
                
    types1 = {k: v for k, v in sorted(types1.items(), key=lambda item: item[1], reverse=True)}
    types2 = {k: v for k, v in sorted(types2.items(), key=lambda item: item[1], reverse=True)}
    
    return types1, types2


def get_event_typing_main(w, pos, thres = 0.0, m = 3, depth = 3):
    if not pos:
        pos = nltk.tag.pos_tag(w.split())[0][1]
    
    types1, types2 = get_event_typing(w, thres, m, depth)
    
    if not (types1 or types2):
        if pos.startswith("V"):    
            w_tmp = WordNetLemmatizer().lemmatize(w, "v")
            types1, types2 = get_event_typing(w_tmp, thres, m, depth)
            if (types1 or types2):
                return types1, types2
            
        if pos.startswith("N"):
            w_tmp = WordNetLemmatizer().lemmatize(w, "n")
            types1, types2 = get_event_typing(w_tmp, thres, m, depth)
            if (types1 or types2):
                return types1, types2
            
        w_tmp = WordNetLemmatizer().lemmatize(w)
        types1, types2 = get_event_typing(w_tmp, thres, m, depth)
        
        if (types1 or types2):
            return types1, types2
        
        stemmer = SnowballStemmer("english")
        w_tmp = stemmer.stem(w)
        types1, types2 = get_event_typing(w_tmp, thres, m, depth)
            
    
    return types1, types2
            
            
# text = "eruption"

# a, b = get_event_typing_main(text, pos="", thres=0.0, m=10, depth=3) 
# print(a)
# print(b)

In [74]:
stops = set(stopwords.words('english'))

text = "The number of people already infected by the mystery virus emerging in China is far greater than official figures suggest, scientists have told the BBC. There have been more than 60 confirmed cases of the new coronavirus, but UK experts estimate a figure nearer 1700. Two people are known to have died from the respiratory illness, which appeared in Wuhan city in December."
tokens = text.lower().split()

pos_tags = nltk.tag.pos_tag(text.split())
for x in pos_tags:
    token = x[0].lower()
    pos = x[1]
    if token not in stops:
        if pos.startswith("V") or pos=="NN":
            a, b = get_event_typing_main(token, pos=pos, thres=0.0, m=10, depth=3) 
            print(token, pos)
            if a:
                print(a)
                print("\n")
            if b:
                print(b)
            print("_"*100)
#             print("\n")


number NN
{':amount:magnitude:property': 0.5358333333333333, ':definite_quantity:measure:abstraction': 0.5358333333333333, ':company:social_gathering:gathering': 0.5358333333333333, ':positive_identification:identification:evidence': 0.5358333333333333, ':size:property:attribute': 0.5358333333333333, ':grammatical_category:class:collection': 0.5358333333333333, ':merchandise:commodity:artifact': 0.5358333333333333, ':name:denote:mean': 0.5358333333333333, ':limit:decrease:change': 0.5358333333333333, ':designate': 0.5}


{'amount': 0.5, 'definite_quantity': 0.5, 'company': 0.5, 'positive_identification': 0.5, 'size': 0.5, 'grammatical_category': 0.5, 'merchandise': 0.5, 'designate': 0.5, 'name': 0.5, 'limit': 0.5, 'magnitude': 0.3333333333333333, 'property': 0.3333333333333333, 'measure': 0.3333333333333333, 'social_gathering': 0.3333333333333333, 'identification': 0.3333333333333333, 'class': 0.3333333333333333, 'commodity': 0.3333333333333333, 'denote': 0.3333333333333333, 'decrease'

In [32]:
WordNetLemmatizer().lemmatize("bombed")

stemmer = SnowballStemmer("english")
print(stemmer.stem("generalized"))
print(stemmer.stem("bombed"))

general
bomb


In [30]:
s = wn.synsets('walks')

for i in s:
#     print(i.definition())
#     x = fn.frames_by_lemma('(?i)' + i.definition() + '(?i)')
    l = i.name().split(".")[0]
#     print(l)
    a = fn.frames_by_lemma("rise_up")
    print(a)

[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]


In [ ]:
x = fn.frames(r'(?i)crim')
x.sort(key=itemgetter('ID'))
[i.name for i in x]
# len(x)

In [ ]:
fn.frames_by_lemma(r'(?i)'+"walk")

In [ ]:
wf = json.load(open("word_fequency.json"))
wfo = {}
c = 0
for t in wf:
    c = c + wf[t]

for t in wf:
    wfo[t] = wf[t]/c
    
    
json_object = json.dumps(wfo)

with open("word_fequency_norm.json", "w") as outfile:
    outfile.write(json_object)

In [ ]:
text = '''New York City Police Department Chief of Detectives James Essig detailed what suspect Frank James did after shooting at passengers on a Brooklyn subway on Tuesday.

"We believe, but this is still early in the investigation, that after firing his weapon 33 times at innocent New York City subway riders, Mr. James boarded an R train that had pulled into the station, went one stop up and exited at the 25th Street station. We also have a picture of that. The gun used in this — a 9 millimeter Glock — which was recovered at this crime scene, was purchased by Mr. James in 2011 in Ohio," Essig said.
The official also noted that the construction helmet James was wearing was recovered in a garbage bin.

Essig said officials then tracked James' location before he was arrested.

"We tracked Mr. James, and his last known whereabouts was 7th Avenue and 9th Street in Park Slope, entering the subway. Minutes ago, thankfully, NYPD patrol officers from the 9th Precinct responded to St. Marks [Place] and First Avenue, where they apprehended him without incident," Essig said, noting that the suspect was in Park Slope at about 9:15 a.m. ET yesterday.

Kenneth Corey, the NYPD chief of department, described how the arrest unfolded today, explaining that investigators got a CrimeStoppers tip that the suspect was in a McDonald's at 6th Street and First Avenue in the East Village.

"Officers respond to the McDonald's. He's not in the McDonald's. They start driving around the neighborhood looking for him, they see him on the corner of St. Marks [Place] and First [Avenue] and they take him into custody," he said.

The case was quickly solved using technology and video canvassing, he added. He also thanked federal and regional partners.


'''

In [ ]:
def trigger_type_prediction = 'None'

            x = fn.frames('(?i)' + tokens[tmp_trigger_position[0]] + '(?i)')
            x.sort(key=itemgetter('ID'))
            if len(x) > 0:
                trigger_type_prediction = x[0]["name"]
            # print("Token: ", tokens[tmp_trigger_position[0]], " , trigger_type_prediction:", trigger_type_prediction)
        
            else:
                x = fn.frames('(?i)' + WordNetLemmatizer().lemmatize(tokens[tmp_trigger_position[0]],'v') + '(?i)')
                # x.sort(key=itemgetter('ID'))
                if len(x) > 0:
                    trigger_type_prediction = x[0]["name"]
                else:
                    event_types_dict = get_event_typing(tokens[tmp_trigger_position[0]], thres = 0.0, m = 3, depth=3)
                    if len(event_types_dict)>0:
                        trigger_type_prediction = [elem for elem in event_types_dict.values()][0]
                    elif len(trigger_type_prediction)==0:
                        if tmp_trigger_position in predicate_sense:
                            x = fn.frames_by_lemma('(?i)' + predicate_sense[tmp_trigger_position] + '(?i)')
                            # x.sort(key=itemgetter('ID'))
                            if len(x) > 0:
                                trigger_type_prediction = x[0]["name"]
                            else:
                                trigger_type_prediction = predicate_sense[tmp_trigger_position]
                        elif tmp_trigger_position in detected_mentions:
                            trigger_type_prediction = detected_mentions[tmp_trigger_position]

In [ ]:
def get_event_typing(w, thres = 0.0, m = 3, depth = 3):
    x = fn.frames('(?i)' + tokens[tmp_trigger_position[0]] + '(?i)')
    
    pos = nltk.tag.pos_tag(w.split())[0]
    print(pos)
    if len(pos)==0:
        return []
    w = pos[0]
    ps = pos[1]
    
        
    
    print(ps)
    
    hyper = lambda s: s.hypernyms()
    ws = wn.synsets(w)
    print(ws)
    types = {}
    for e in ws:
        if e.name().split(".")[0]==w:
            tmp_event_types = list(e.closure(hyper))
            t = 0
            c = 0 
            i = 0
            tmp_types = ""
            for tmp_event in tmp_event_types:
                d = wn.path_similarity(e, tmp_event)
        #         print(tmp_event, d)
                b = m**i
                t = t + d/b
                if d and (d>=thres/b):
                    tmp_types = tmp_types + ":" + tmp_event.name().split(".")[0]
                    c = c + 1
                    if c >= depth:
                        break
                i = i + 1
            if len(tmp_types) > 0:
                types[t] = tmp_types
        #     print("\n")


    types = {k: v for k, v in sorted(types.items(), key=lambda item: item[0], reverse=True)}
    
    return types

In [ ]:
get_event_typing("slept", thres = 0.0, m = 3, depth = 3)

In [ ]:


filtered_words = set(stopwords.words('english'))

wfo = json.load(open("word_fequency_NORM.json"))

# text = "The eruption of an underwater volcano near Tonga on Saturday was likely the biggest recorded anywhere on the planet in more than 30 years, according to experts. Dramatic images from space captured the eruption in real time, as a huge plume of ash, gas and steam was spewed up to 20 kilometers into the atmosphere and tsunami waves were sent crashing across the Pacific."
# text = "At least seven historically Black colleges and universities (HBCUs) across the United States received back-to-back bomb threats this week, forcing students to evacuate or shelter in place while authorities investigated. The threats come amid a dramatic rise in bombings in the US and follow bomb threats at other US colleges last November."
# text = "Crews fighting a massive fire along the central coast of California near the iconic Highway 1 made progress Sunday in containing the blaze, but dozens of homes remained under evacuation orders. The Colorado fire ignited Friday evening in Palo Colorado Canyon in the Big Sur region of Monterey County and swelled to 1050 acres Saturday, up from 100 acres a day prior."

text = re.sub('\n+',' ', text)
text = re.sub('\t+',' ', text)
text = re.sub('\s+',' ', text)

number_of_sentences = len(sent_tokenize(text))


# print(text)
print("Number of Sentences: ", number_of_sentences)
print("Number of Tokens: ", len(word_tokenize(text)))
print("Number of Characters: ", len(text))

print("\n")


verb_max = 0.00001
noun_max = 0.00005

start_time = time()
pos = nltk.tag.pos_tag(text.split())


for p in pos:
    event_types = []
    w = p[0].lower()
    ps = p[1]
#     print(p[0], ps)
    if ps =="NNS" or ps =="NNP":
        continue
    if ((not ps=="NN") and (not ps.startswith("VB"))) or ((not (p[0] in words.words())) or (not (p[0] in wf))):
        tmp_w = WordNetLemmatizer().lemmatize(w,'v')
        if tmp_w and (tmp_w is not w):
            w = tmp_w
            ps = nltk.tag.pos_tag(w.split())[0][1]
        else:
            tmp_w = WordNetLemmatizer().lemmatize(w,'n')
            if tmp_w and (tmp_w is not w):
                w = tmp_w
                ps = nltk.tag.pos_tag(w.split())[0][1]
    if ps=="NN" or ps.startswith("VB"):
        if (not w in filtered_words) and (w in words.words()) and (w in wfo):
            if ps=="NN" and wfo[w] > noun_max:
                continue
            if ps.startswith("VB") and wfo[w] > verb_max:
                continue
            if p[0].lower() in wfo:
                if ps=="NN" and wfo[p[0].lower()] > noun_max:
                    continue
                if ps.startswith("VB") and wfo[p[0].lower()] > verb_max:
                    continue
            event_types_dict = get_event_typing(w, thres = 0.0, m = 3, depth=3)
            event_types = [elem for elem in event_types_dict.values()]
            if len(event_types)==0:
                print("f")
                f = fn.frames(p[0].lower())
                if len(f) > 0:
                    event_types = [t["name"] for t in f]
                else:
                    f = fn.frames_by_lemma(p[0].lower())
                    if len(f) > 0:
                        event_types = [t["name"] for t in f]
                    else:
                        f = fn.frames(w)
                        if len(f) > 0:
                            event_types = [t["name"] for t in f]
                        else:
                            f = fn.frames_by_lemma(w)
                            if len(f) > 0:
                                event_types = [t["name"] for t in f]
            print(p[0], event_types, wfo[w], ps, w)
#             print(p[0])
#             print(w)


total_time = time() - start_time
avg_time = total_time/number_of_sentences
print("\n")
print("Total Time: ", total_time)
print("Average Time per Sentence: ", avg_time)
                

In [ ]:
suspect 
shooting - 
passanger 
subway
investigation 
firing - 
weapon 
crime 
purchased
tracked -
suspect 
arrest 
suspect
case 
solved

In [ ]:
import nltk
a ="reported believed regarding  has disputed was according yesterday confidence version events missile strike warship source intelligence"

pos = nltk.tag.pos_tag("yesterday".split())
pos